### Student Name: Sachid Deshmukh
### Data 612 Recommender System Assignment-1
### Date 06/08/2019

### We are developing a recommender system which will recommend Books to users. In this exercise we are leveraging a toy dataset which contains five users and their ratings to five books

In [137]:
import numpy as np
import pandas as pd
import random

### Build toy dataset

In [138]:
splitFlag = np.array([0,1,1,1,1,1,1,0,0,1,1,1,1,0,1,0,1,1,1,1,1,1,0,0,1]).reshape(5,5)
name = ["John", "Daisy", "Amit", "Abrahaim", "Peter"]
movie1 = list(np.random.randint(3,6,5))
movie2 = list(np.random.randint(3,6,5))
movie3 = list(np.random.randint(1,6,5))
movie4 = list(np.random.randint(4,6,5))
movie5 = list(np.random.randint(2,6,5))
data = pd.DataFrame(zip(name, movie1, movie2, movie3, movie4, movie5), columns=["User","Signal and the Noise", "Data Science for Business", "Automated Data Collection with R", "R for Data Science", "Introduction to Statistical Learning"])
data.head(10)


,User,Signal and the Noise,Data Science for Business,Automated Data Collection with R,R for Data Science,Introduction to Statistical Learning
0,John,5,4,4,5,3
1,Daisy,4,4,1,4,5
2,Amit,3,3,5,5,4
3,Abrahaim,3,5,3,4,3
4,Peter,5,3,3,5,2


### Create a toy dataset and split it into train and test set

In [139]:

data_train = pd.DataFrame(index = data.index, columns = data.columns)
data_train['User'] = data['User']
data_test = pd.DataFrame(index = data.index, columns = data.columns)
data_test['User'] = data['User']
for i in range(data.shape[0]):
    for j in range(1,data.shape[1]):
        if splitFlag[i][j-1]:
            data_train.iloc[i][j] = data.iloc[i][j]
        else:
            data_test.iloc[i][j] = data.iloc[i][j]


In [140]:
data_train.head()
#data_test.head()

,User,Signal and the Noise,Data Science for Business,Automated Data Collection with R,R for Data Science,Introduction to Statistical Learning
0,John,NaN,4,4,5,3
1,Daisy,4,4,NaN,NaN,5
2,Amit,3,3,5,NaN,4
3,Abrahaim,NaN,5,3,4,3
4,Peter,5,3,NaN,NaN,2


### Calculate Raw average on training dataset

In [141]:
def calcRawAvg(df):
    movieSum = 0
    movieCount = 0
    for i in range(df.shape[0]):
        for j in range(1, df.shape[1]):
            rating = df.iloc[i][j]
            if np.isnan(rating) == False:
                movieSum = movieSum + rating
                movieCount = movieCount + 1
    rawAvg = movieSum/movieCount
    return rawAvg

rawAvg = calcRawAvg(data_train)
rawAvg




3.8333333333333335

### Calculate test and train RMSE using Raw average

In [142]:
def calcRMSE(df, rawAvg):
    residuals = []
    for i in range(df.shape[0]):
        for j in range(1, df.shape[1]):
            rating = df.iloc[i][j]
            if np.isnan(rating) == False:
                residuals.append((rating - rawAvg)**2)
    return(np.sqrt(sum(residuals)/len(residuals)))

trainRMSE = calcRMSE(data_train, rawAvg)
print('Train RMSE = %s' %(trainRMSE))
testRMSE = calcRMSE(data_test, rawAvg)
print('Test RMSE = %s' %(testRMSE))

Train RMSE = 0.8975274678557507
Test RMSE = 1.39015815019


### Calculate Baseline Predictors for each user and movie

In [143]:
data_train.head()
data_train_melt = data_train.melt(id_vars=['User'], value_vars=['Signal and the Noise', 'Data Science for Business', 'Automated Data Collection with R', 'R for Data Science', 'Introduction to Statistical Learning'], var_name='Movies', value_name = "Ratings")
data_train_melt.head(100)
data_train_melt = data_train_melt.dropna()
user_mean = pd.DataFrame(data_train_melt.groupby('User')['Ratings'].mean(numeric_only=False))
movie_mean = pd.DataFrame(data_train_melt.groupby('Movies')['Ratings'].mean(numeric_only=False))
user_bias = user_mean['Ratings'].apply(lambda x: x-rawAvg)
user_mean['Bias'] = user_bias

movie_bias = movie_mean['Ratings'].apply(lambda x: x-rawAvg)
movie_mean['Bias'] = movie_bias
movie_mean.head()
#user_mean.head()



,Ratings,Bias
Movies,,
Automated Data Collection with R,4.0,0.166667
Data Science for Business,3.8,-0.033333
Introduction to Statistical Learning,3.4,-0.433333
R for Data Science,4.5,0.666667
Signal and the Noise,4.0,0.166667


### Calculate test and train RMSE using Baseline predictors

In [144]:
def suggestRatings(dfRow):
    if dfRow['Ratings'] == 0:
        return 0
    else:
        basePred = rawAvg + user_mean.loc[dfRow['User']]['Bias'] + movie_mean.loc[dfRow['Movies']]['Bias']
        if basePred < 1:
            return 1
        elif basePred > 5:
            return 5
        else:
            return basePred
data_test_melt = data_test.melt(id_vars=['User'], value_vars=['Signal and the Noise', 'Data Science for Business', 'Automated Data Collection with R', 'R for Data Science', 'Introduction to Statistical Learning'], var_name='Movies', value_name = "Ratings")
data_test_melt = data_test_melt.dropna()

baseline_Pred_Test = data_test_melt.apply(suggestRatings, axis=1)
data_test_melt['BaselinePred'] = baseline_Pred_Test

baseline_Pred_Train = data_train_melt.apply(suggestRatings, axis=1)
data_train_melt['BaselinePred'] = baseline_Pred_Train
data_test_melt.head(100)

baseline_train_residuals = np.array((data_train_melt['Ratings'] - data_train_melt['BaselinePred'])**2)
baseline_train_rmse = np.sqrt(sum(baseline_train_residuals)/len(baseline_train_residuals))

baseline_test_residuals = np.array((data_test_melt['Ratings'] - data_test_melt['BaselinePred'])**2)
baseline_test_rmse = np.sqrt(sum(baseline_test_residuals)/len(baseline_test_residuals))
print('Baseline Train RMSE = %2.2f' %baseline_train_rmse)
print('Baseline Test RMSE = %2.2f' %baseline_test_rmse)

Baseline Train RMSE = 0.78
Baseline Test RMSE = 1.53


### With Raw Average we got Train RMSE of 0.89 and Test RMSE of 1.39. With Baseline predictors we got train RMSE of 0.78 and test RMSE of 1.59

### Between Baseline predictros and Raw average there is an improvement of (1- (0.78/0.89)) * 100 = 13% in train rmse

### Between Baseline predictors and Raw average there is an improvement of (1- (1.53/1.39)) * 100  = -10% in test rmse

### Baseline predictors are performing good on training data but not in the test data. This indicates that model is overfitting to training data set. This is expected since our dataset is so small. If we get larger dataset we can overcome overfitting and we would see that baseline predictors are better predictors on both training and test dataset